In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def compute_metrics(pred,ground_labels):
    labels_all = ground_labels
    preds_all = list(pred)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels_all, preds_all)
    acc = accuracy_score(labels_all, preds_all)
    confusion_mat = confusion_matrix(labels_all, preds_all)
    # tn, fp, fn, tp = confusiton_mat.ravel()
    out_dict = {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'confusiton_mat': confusion_mat
    }
    return out_dict

In [ ]:
import pandas as pd
import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support, \
    confusion_matrix
import torch
import time
import numpy as np
from sklearn.model_selection import train_test_split

import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dfLiarSementic = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Results/LIAR/Liar_sementic.csv")
dfLiarSentiment = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Results/LIAR/LIAR_predictions_emotions.csv")
dfLiarLexicon = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Results/LIAR/Liar_lexicon.csv")

In [ ]:
dff=dfLiarSentiment.merge(dfLiarSementic, how='inner', on='ID',suffixes=('_Sentiment', '_Sementic'))
df=dff.merge(dfLiarLexicon, how='inner', on='ID',suffixes=('', '_Lexicon'))
df=df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)

In [ ]:
df["qn_symbol_per_sentence"]=df["qn_symbol"]/df["num_sentences"]
df["num_exclamation_per_sentence"]=df["num_exclamation"]/df["num_sentences"]
df["url_count_per_sentence"]=df["url_count"]/df["num_sentences"]

In [ ]:
sementic_features= [
                    'url_count',
                    'qn_symbol',
                    'num_chars',
                    'num_words',
                    'num_sentences',
                    'words_per_sentence',
                    'characters_per_word',
                    'punctuations_per_sentence',
                    'num_exclamation',
                    'get_sentiment_polarity',
                    'lexical_diversity',
                    'content_word_diversity',
                    'redundancy',
                    'noun',
                    'verb',
                    'adj',
                    'adv',
                    "qn_symbol_per_sentence",
                    "num_exclamation_per_sentence",
                    "url_count_per_sentence"
                    ]

LangMod_Features=   [
                     'fake_score', 
                    'true_score', 
                    'common_score'
                    ]

Sentiment_features= [
                    #  'highest_eight_label', 
                    'anger', 
                    'anticipation',
                    'disgust', 
                    'fear',
                    'joy', 
                    'sadness', 
                    'surprise', 
                    'trust'
                    ]
All_features=sementic_features+LangMod_Features+Sentiment_features

In [ ]:
df=df.loc[df["lang"]=="en"]

In [ ]:
df["label"]=df["label_Sentiment"]

In [ ]:
df=df[All_features+["label","ID","split_Sentiment"]]

In [ ]:
df["label"] = df["label"].replace(["true","mostly-true","half-true"],0)
df["label"] = df["label"].replace(["false","barely-true","pants-fire"],1)

In [ ]:
df.label.value_counts()

0    7088
1    5596
Name: label, dtype: int64

In [ ]:
df_train = df.loc[df["split_Sentiment"]=="train"][All_features+['label']]
df_test = df.loc[df["split_Sentiment"]!="train"][All_features+['label']]
# df_valid = df.loc[df["split_Sentiment"]=="valid"][All_features+['label']]

In [ ]:
df_train.label.value_counts()

0    5712
1    4438
Name: label, dtype: int64

# H2o.ai


In [ ]:
! pip install h2o -q
import h2o 
from h2o.automl import H2OAutoML

In [ ]:
h2o.init()

In [ ]:
hf_train = h2o.H2OFrame(df_train)
hf_valid = h2o.H2OFrame(df_valid)

cat_feat = ['label', 'highest_eight_label']
hf_train[cat_feat] = hf_train[cat_feat].asfactor()
hf_valid[cat_feat] = hf_valid[cat_feat].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
x = hf_train.columns
x.remove('label')

aml_h1n1 = H2OAutoML(
    max_models = 25, 
    seed = 10, 
    verbosity="info", 
    # exclude_algos = ["DeepLearning"],
    balance_classes = True,
    max_after_balance_size = 1.5,
    max_runtime_secs_per_model = 120,
    # nfolds=0
)

aml_h1n1.train(x = x, y = 'label', training_frame = hf_train, validation_frame=hf_valid)

In [ ]:
hf_test = h2o.H2OFrame(df_test)

cat_feat = ['label', 'highest_eight_label']
hf_test[cat_feat] = hf_test[cat_feat].asfactor()

In [ ]:
pred=aml_h1n1.leader.predict(hf_test)

predictions = pred.as_data_frame()

predictions.head()

In [ ]:
y_pred = predictions['predict'].tolist()
y_true = df_test['label'].tolist()

accuracy_score(y_true, y_pred)

0.43626286619160726

# TPOT


In [ ]:
# http://epistasislab.github.io/tpot/using/

In [ ]:
!pip install TPOT -q

     |████████████████████████████████| 87 kB 2.8 MB/s 
     |████████████████████████████████| 160 kB 26.5 MB/s 
     |████████████████████████████████| 166.7 MB 15 kB/s 


In [ ]:
from tpot import TPOTClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
X_train=df_train[All_features]
y_train=df_train["label"]
X_val=df_test[All_features]
y_val=df_test["label"]

## recall

In [ ]:
# Create a tpot optimizer with parameters
tpot = TPOTClassifier(scoring = 'recall', 
                      config_dict='TPOT light' ,#TPOT light ,TPOT cuML
                    #  max_time_mins = 60, 
                     n_jobs = -1,
                     verbosity = 2,
                     periodic_checkpoint_folder="/content/drive/Shareddrives/[FYP] Fake News Detection/Feature Fusion/tPOT/Liar",
                    # memory="auto"
                    )

In [ ]:
tpot.fit(X_train,y_train)

Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.536730247722356

Generation 2 - Current best internal CV score: 0.6237189839219152

Generation 3 - Current best internal CV score: 0.6237189839219152

Generation 4 - Current best internal CV score: 1.0

Generation 5 - Current best internal CV score: 1.0

Generation 6 - Current best internal CV score: 1.0

Generation 7 - Current best internal CV score: 1.0

Generation 8 - Current best internal CV score: 1.0

Generation 9 - Current best internal CV score: 1.0

Generation 10 - Current best internal CV score: 1.0

Generation 11 - Current best internal CV score: 1.0

Generation 12 - Current best internal CV score: 1.0

Generation 13 - Current best internal CV score: 1.0

Generation 14 - Current best internal CV score: 1.0

Generation 15 - Current best internal CV score: 1.0

Generation 16 - Current best internal CV score: 1.0

Generation 17 - Current best internal CV score: 1.0

Generation 18 - Current best internal CV score: 1.0

Generation

TPOTClassifier(config_dict='TPOT light', crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=100,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder='/content/drive/Shareddrives/[FYP] '
                                          'Fake News Detection/Feature '
                                          'Fusion/tPOT/Liar',
               population_size=100, random_state=None, scoring='recall',
               subsample=1.0, template=None, use_dask=False, verbosity=2,
               warm_start=False)

In [ ]:
# Best pipeline: BernoulliNB(Binarizer(MinMaxScaler(input_matrix), threshold=1.0), alpha=0.001, fit_prior=False)
# TPOTClassifier(config_dict='TPOT light', crossover_rate=0.1, cv=5,
#                disable_update_check=False, early_stop=None, generations=100,
#                log_file=None, max_eval_time_mins=5, max_time_mins=None,
#                memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
#                periodic_checkpoint_folder='/content/drive/Shareddrives/[FYP] '
#                                           'Fake News Detection/Feature '
#                                           'Fusion/tPOT/Liar',
#                population_size=100, random_state=None, scoring='recall',
#                subsample=1.0, template=None, use_dask=False, verbosity=2,
#                warm_start=False)


In [ ]:
# Export the pipeline as a python script file
# tpot.export('/content/drive/Shareddrives/[FYP] Fake News Detection/Feature Fusion/tPOT/tpot_exported_pipeline1.py')

In [ ]:
tpot.score(X_val, y_val)

0.9991364421416234

In [ ]:
pred=tpot.predict(X_val)

In [ ]:
gg=compute_metrics(pred,y_val)
d=gg
tn, fp, fn, tp = d["confusiton_mat"].ravel()
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp'))  # correct
print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp))

prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp
0.000	0.457	0.000	0.999	0.000	0.627	0.457	0	1376	1	1157


## accuracy

In [ ]:
# Create a tpot optimizer with parameters
tpot = TPOTClassifier(scoring = 'accuracy', #accuracy ,f1 ,recall
                      config_dict='TPOT light' ,#TPOT light ,TPOT cuML
                    #  max_time_mins = 60, 
                     n_jobs = -1,
                     verbosity = 2,
                     periodic_checkpoint_folder="/content/drive/Shareddrives/[FYP] Fake News Detection/Feature Fusion/tPOT/Liar_accuracy",
                    # memory="auto"
                    )

In [ ]:
tpot.fit(X_train,y_train)

Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.5784236453201971

Generation 2 - Current best internal CV score: 0.5803940886699508

Generation 3 - Current best internal CV score: 0.5807881773399014

Generation 4 - Current best internal CV score: 0.580985221674877

Generation 5 - Current best internal CV score: 0.5813793103448275

Generation 6 - Current best internal CV score: 0.581871921182266

Generation 7 - Current best internal CV score: 0.5820689655172414

Generation 8 - Current best internal CV score: 0.5820689655172414

Generation 9 - Current best internal CV score: 0.5828571428571429

Generation 10 - Current best internal CV score: 0.5828571428571429

Generation 11 - Current best internal CV score: 0.5828571428571429

Generation 12 - Current best internal CV score: 0.5866009852216749

Generation 13 - Current best internal CV score: 0.5866009852216749

Generation 14 - Current best internal CV score: 0.5866009852216749

Generation 15 - Current best internal CV score: 0.59004926

TPOTClassifier(config_dict='TPOT light', crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=100,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder='/content/drive/Shareddrives/[FYP] '
                                          'Fake News Detection/Feature '
                                          'Fusion/tPOT/Liar_accuracy',
               population_size=100, random_state=None, scoring='accuracy',
               subsample=1.0, template=None, use_dask=False, verbosity=2,
               warm_start=False)

In [ ]:
Best pipeline: LogisticRegression(StandardScaler(RobustScaler(RBFSampler(MinMaxScaler(MinMaxScaler(input_matrix)), gamma=0.05))), C=20.0, dual=False, penalty=l2)
TPOTClassifier(config_dict='TPOT light', crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=100,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder='/content/drive/Shareddrives/[FYP] '
                                          'Fake News Detection/Feature '
                                          'Fusion/tPOT/Liar_accuracy',
               population_size=100, random_state=None, scoring='accuracy',
               subsample=1.0, template=None, use_dask=False, verbosity=2,
               warm_start=False)

In [ ]:
# Export the pipeline as a python script file
tpot.export('/content/drive/Shareddrives/[FYP] Fake News Detection/Feature Fusion/tPOT/tpot_exported_pipeline2.py')

In [ ]:
tpot.score(X_val, y_val)

0.569060773480663

In [ ]:
pred=tpot.predict(X_val)

In [ ]:
gg=compute_metrics(pred,y_val)
d=gg
tn, fp, fn, tp = d["confusiton_mat"].ravel()
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp'))  # correct
print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp))

prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp
0.575	0.552	0.795	0.301	0.667	0.389	0.569	1094	282	810	348


# Rapids

https://medium.com/@kayush206/rapids-on-google-colab-d11df50abcf3

In [ ]:
!nvidia-smi # need Tesla T4

Sun Oct 10 03:04:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc -V #>=9.2

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!python -V; pip -V  #3.6

Python 3.7.12
pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [ ]:
# installing cuml dependencies
!apt install libopenblas-base libomp-dev 
# installing the cuml 
!pip install cuml-cuda100

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-base is already the newest version (0.2.20+ds-4).
libopenblas-base set to manually installed.
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 2s (143 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1

If you try to import cudf at this point, it will give you, librmm.so not found error.
- copy this librmm.so to your current working directory:

In [ ]:
!cp /usr/local/lib/python3.6/dist-packages/libcuml.so /usr/lib64-nvidia/

Step 3: Here you have to set the path for NVVM. copy and paste the below lines to notebook cell.

In [ ]:
import os  
os.environ['NUMBAPRO_NVVM']='/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so'  
os.environ['NUMBAPRO_LIBDEVICE']='/usr/local/cuda-10.0/nvvm/libdevice'